# load data

In [1]:
# import load_iris function from datasets module
from sklearn.datasets import load_iris
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import random
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier

/home/milad/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data = pd.read_csv("../../../accuracy_drop_proj/datasets/EEG_data_Epileptic_Seizure_Recognition_test.csv")
# data = data.drop(['Unnamed: 0'], axis=1)
data.shape



(99, 179)

In [ ]:
y = data.y.values

del data["y"] # remove rings from data, so we can convert all the dataframe to a numpy 2D array.
X = data.values.astype(np.float)

In [ ]:
y = data['y']
# data = data.drop(['y'],axis=1)
# data.head()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data, y, test_size=0.25,random_state=4)

In [ ]:
#find the order of the feature according to information gain
model = ExtraTreesClassifier()
model.fit(data, y)


information_gain = {}
for i in range(len(model.feature_importances_)):
    information_gain.update({i: model.feature_importances_[i]})


In [ ]:
col_sorted=sorted(information_gain.items(), key=lambda x:x[1],reverse=True)
select_col=[]
for i in col_sorted:
    select_col.append(i[0])

In [ ]:
print(select_col)

In [3]:
temp = data.iloc[:,[9, 52, 168, 33, 157, 107, 159, 1, 54, 166,178]]

In [4]:
temp.head()

,X10,X53,X169,X34,X158,X108,X160,X2,X55,X167,y
0,-38,101,8,-96,-53,43,-72,190,73,-11,4
1,232,285,168,314,63,200,89,382,215,131,1
2,-94,-40,29,-13,80,5,23,-39,-31,-42,5
3,-79,-82,-80,-67,-38,-87,-29,-101,-93,-74,5
4,-59,-3,10,-41,11,57,-1,-65,39,15,5


In [5]:
y = temp.y.values

del temp["y"] # remove rings from data, so we can convert all the dataframe to a numpy 2D array.
X = temp.values.astype(np.float)

In [ ]:
y = temp['y']
temp = temp.drop(['y'],axis=1)
temp.shape

In [6]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=4)

In [20]:
from sklearn import svm

model_svm= svm.SVC(probability=True)
model_svm.fit(x_train, y_train)


y_predsvm = model_svm.predict(x_test)
print('Accuracy of svm classifier on train set: {:.2f}'.format(accuracy_score(y_test,y_predsvm)))


Accuracy of svm classifier on train set: 0.20


In [9]:
from sklearn import tree
model1=tree.DecisionTreeClassifier()
print("train tree")


model1.fit(x_train,y_train)
y_pred1 = model1.predict(x_test)
print('Accuracy of tree classifier on train set: {:.2f}'.format(accuracy_score(y_test,y_pred1)))

train tree
Accuracy of tree classifier on train set: 0.24


In [ ]:
model_svm.predict([[-14,-61,-29,-1,44,18,-45,-14,1,16]])

In [25]:
import heapq
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import chi2
from sklearn.ensemble import ExtraTreesClassifier
from accuracy_drop_proj.strategies.change_combination.change_combination import Change_Combination

"""
this method according the user request find the difference of the probability for each class and then sort the rows according to them.
for example if we have [0.1  0.6  0.3] and request of user be [0,1] we compute 0.5 for this row and sort the row Ascending
"""

class Change_ProbabilityDistance_RankFeature(object):
    def __init__(self):
        pass

    def change(self,x_train, y_train, percetage, mnb, change_plan):
        number_change_requested = int(percetage / 100 * x_train.shape[0])
        print("{} percentage error is equal to {} change \n".format(percetage, number_change_requested))


        used_row ={}
        occurred_change = 0
        all_changed = 1
        change_done = False
        x_train_changed = np.copy(x_train)

        #---------------------find the order of the feature according to information gain-----------------------

        model = ExtraTreesClassifier()
        model.fit(x_train, y_train)

        print("combination of feature")

        information_gain = {}
        for i in range(len(model.feature_importances_)):
            information_gain.update({i: model.feature_importances_[i]})

        ranked_information_dic = {}
        sum_gain = 0
        for L in range(0,x_train.shape[1] + 1):
            for subset in Change_Combination.combinations_index(self,information_gain.keys(), L):
                if not subset:
                    pass
                else:

                    for key in subset:
                        sum_gain = sum_gain + information_gain.get(key)
                    ranked_information_dic.update({tuple(subset): sum_gain})
                    sum_gain = 0

        print("create all subset")

        all_subset = sorted(ranked_information_dic.items(), key=lambda item: len(item[0]) * 1000 - item[1], reverse=False)




        probability = mnb.predict_proba(x_train)
        probability_distance={}

        #----------------------------------------------changing--------------------------------------------------

        for i in range(len(change_plan["key"])):
            occurred_change = 0

            indices = [t for t, x in enumerate(y_train) if x == change_plan["key"][i][0]]
            # print(indices)
            print("{} rows have target {} \n".format(len(indices), change_plan["key"][i][0]))

            probability_distance.clear()
            probability_distance_sorted=[]

            # find the distance probability between the class that user need to change

            for elements in indices:
                probability_distance.update({elements:np.abs(probability[elements][change_plan["key"][i][0]]- probability[elements][change_plan["key"][i][1]])})
            # ---------------------------finding the order of the row according to probability distance-------------------------
            # sort the row according the distance probability

            probability_distance_sorted = sorted(probability_distance.items(), key=lambda x: x[1], reverse=False)
            indices=[]
            for j in probability_distance_sorted:
                indices.append(j[0])

            print(indices)

            print("try in indices")
            for p in range(len(indices)):

                if (all_changed == number_change_requested + 1):
                    print("your requests have been done :)")
                    break
                    
               
            
#                 print(mnb.predict([x_train[indices[p]]]))
                if y_train[indices[p]] == mnb.predict([x_train[indices[p]]]) and indices[p] not in used_row:

                    change_done = False
                    
                    for subset in all_subset:
                        if change_done:
                            break
                        else:

                            if (occurred_change == change_plan["number"][i]):
                                #                         print("part of your request has been done :))))")
                                break

                            print("try to change, with changing index {} on row {}".format(list(subset[0]),indices[p]))

                            #######################################################
                            # impose Outlier insted of 0

                           ########################################################
                            x_train_changed[indices[p]][list(subset[0])] = 0

                            if (change_plan["key"][i][1] == mnb.predict([x_train_changed[indices[p]]])[0]):

                                print(x_train[indices[p]], mnb.predict([x_train[indices[p]]])[0])
                                print(x_train_changed[indices[p]],
                                      mnb.predict([x_train_changed[indices[p]]])[0])
                                print(" \n change number {} \n".format(all_changed))

                                used_row.update({indices[p]: indices[p]})
                                occurred_change = occurred_change + 1
                                change_done = True
                                all_changed = all_changed + 1
                                break

                            else:
                                x_train_changed[indices[p]] = np.copy(x_train[indices[p]])

        if (all_changed <= number_change_requested):
            print("your request doesn't complete! please change your plan")
        return np.copy(x_train_changed)


In [34]:
change_plan={"key":[[4,2]],"number":[1]}

In [ ]:
confus

In [35]:
classmodel = Change_ProbabilityDistance_RankFeature()
classmodel.change(x_test, y_test,10,model_svm, change_plan)

10 percentage error is equal to 2 change 

combination of feature
create all subset
5 rows have target 4 

[4, 6, 14, 22, 24]
try in indices
try to change, with changing index [5] on row 4
try to change, with changing index [7] on row 4
try to change, with changing index [1] on row 4
try to change, with changing index [8] on row 4
try to change, with changing index [3] on row 4
try to change, with changing index [0] on row 4
try to change, with changing index [6] on row 4
try to change, with changing index [4] on row 4
try to change, with changing index [2] on row 4
try to change, with changing index [9] on row 4
try to change, with changing index [5, 7] on row 4
try to change, with changing index [1, 5] on row 4
try to change, with changing index [1, 7] on row 4
try to change, with changing index [5, 8] on row 4
try to change, with changing index [3, 5] on row 4
try to change, with changing index [7, 8] on row 4
try to change, with changing index [3, 7] on row 4
try to change, with ch

try to change, with changing index [0, 4, 6, 7, 8] on row 4
try to change, with changing index [0, 1, 4, 5, 9] on row 4
try to change, with changing index [1, 2, 3, 7, 9] on row 4
try to change, with changing index [0, 2, 5, 6, 8] on row 4
try to change, with changing index [2, 3, 4, 7, 8] on row 4
try to change, with changing index [1, 4, 5, 6, 9] on row 4
try to change, with changing index [0, 5, 6, 8, 9] on row 4
try to change, with changing index [3, 4, 7, 8, 9] on row 4
try to change, with changing index [0, 1, 3, 6, 8] on row 4
try to change, with changing index [2, 3, 5, 8, 9] on row 4
try to change, with changing index [0, 3, 4, 6, 7] on row 4
try to change, with changing index [0, 1, 2, 4, 7] on row 4
try to change, with changing index [0, 2, 3, 5, 6] on row 4
try to change, with changing index [1, 2, 4, 6, 7] on row 4
try to change, with changing index [0, 1, 4, 7, 9] on row 4
try to change, with changing index [0, 3, 5, 6, 9] on row 4
try to change, with changing index [0, 2

try to change, with changing index [4, 5] on row 6
try to change, with changing index [1, 8] on row 6
try to change, with changing index [4, 7] on row 6
try to change, with changing index [2, 5] on row 6
try to change, with changing index [5, 9] on row 6
try to change, with changing index [1, 3] on row 6
try to change, with changing index [2, 7] on row 6
try to change, with changing index [7, 9] on row 6
try to change, with changing index [3, 8] on row 6
try to change, with changing index [0, 1] on row 6
try to change, with changing index [1, 6] on row 6
try to change, with changing index [1, 4] on row 6
try to change, with changing index [0, 8] on row 6
try to change, with changing index [6, 8] on row 6
try to change, with changing index [0, 3] on row 6
try to change, with changing index [3, 6] on row 6
try to change, with changing index [1, 2] on row 6
try to change, with changing index [1, 9] on row 6
try to change, with changing index [4, 8] on row 6
try to change, with changing in

try to change, with changing index [0, 1, 4, 5, 7, 8] on row 6
try to change, with changing index [1, 4, 5, 6, 7, 8] on row 6
try to change, with changing index [0, 1, 3, 4, 5, 7] on row 6
try to change, with changing index [1, 3, 4, 5, 6, 7] on row 6
try to change, with changing index [0, 3, 5, 6, 7, 8] on row 6
try to change, with changing index [0, 1, 2, 5, 7, 8] on row 6
try to change, with changing index [1, 2, 5, 6, 7, 8] on row 6
try to change, with changing index [0, 1, 5, 7, 8, 9] on row 6
try to change, with changing index [1, 5, 6, 7, 8, 9] on row 6
try to change, with changing index [0, 1, 2, 3, 5, 7] on row 6
try to change, with changing index [1, 2, 3, 5, 6, 7] on row 6
try to change, with changing index [0, 1, 3, 5, 7, 9] on row 6
try to change, with changing index [0, 3, 4, 5, 7, 8] on row 6
try to change, with changing index [1, 3, 5, 6, 7, 9] on row 6
try to change, with changing index [3, 4, 5, 6, 7, 8] on row 6
try to change, with changing index [1, 2, 4, 5, 7, 8] o

try to change, with changing index [0, 2, 5, 8] on row 14
try to change, with changing index [1, 6, 7, 9] on row 14
try to change, with changing index [4, 6, 7, 8] on row 14
try to change, with changing index [1, 4, 5, 9] on row 14
try to change, with changing index [2, 5, 6, 8] on row 14
try to change, with changing index [0, 5, 8, 9] on row 14
try to change, with changing index [0, 1, 3, 8] on row 14
try to change, with changing index [0, 3, 4, 7] on row 14
try to change, with changing index [5, 6, 8, 9] on row 14
try to change, with changing index [1, 3, 6, 8] on row 14
try to change, with changing index [0, 2, 3, 5] on row 14
try to change, with changing index [3, 4, 6, 7] on row 14
try to change, with changing index [1, 2, 4, 7] on row 14
try to change, with changing index [2, 3, 5, 6] on row 14
try to change, with changing index [0, 3, 5, 9] on row 14
try to change, with changing index [0, 2, 7, 8] on row 14
try to change, with changing index [1, 4, 7, 9] on row 14
try to change,

try to change, with changing index [0, 1, 2, 5, 7, 8, 9] on row 14
try to change, with changing index [1, 2, 5, 6, 7, 8, 9] on row 14
try to change, with changing index [0, 1, 2, 3, 5, 7, 9] on row 14
try to change, with changing index [0, 2, 3, 4, 5, 7, 8] on row 14
try to change, with changing index [1, 2, 3, 5, 6, 7, 9] on row 14
try to change, with changing index [2, 3, 4, 5, 6, 7, 8] on row 14
try to change, with changing index [0, 3, 4, 5, 7, 8, 9] on row 14
try to change, with changing index [3, 4, 5, 6, 7, 8, 9] on row 14
try to change, with changing index [1, 2, 4, 5, 7, 8, 9] on row 14
try to change, with changing index [0, 1, 2, 4, 5, 6, 7] on row 14
try to change, with changing index [0, 1, 4, 5, 6, 7, 9] on row 14
try to change, with changing index [1, 2, 3, 4, 5, 7, 9] on row 14
try to change, with changing index [0, 2, 3, 5, 7, 8, 9] on row 14
try to change, with changing index [0, 1, 3, 4, 5, 6, 8] on row 14
try to change, with changing index [2, 3, 5, 6, 7, 8, 9] on ro

try to change, with changing index [0, 1, 4, 5, 9] on row 22
try to change, with changing index [1, 2, 3, 7, 9] on row 22
try to change, with changing index [0, 2, 5, 6, 8] on row 22
try to change, with changing index [2, 3, 4, 7, 8] on row 22
try to change, with changing index [1, 4, 5, 6, 9] on row 22
try to change, with changing index [0, 5, 6, 8, 9] on row 22
try to change, with changing index [3, 4, 7, 8, 9] on row 22
try to change, with changing index [0, 1, 3, 6, 8] on row 22
try to change, with changing index [2, 3, 5, 8, 9] on row 22
try to change, with changing index [0, 3, 4, 6, 7] on row 22
try to change, with changing index [0, 1, 2, 4, 7] on row 22
try to change, with changing index [0, 2, 3, 5, 6] on row 22
try to change, with changing index [1, 2, 4, 6, 7] on row 22
try to change, with changing index [0, 1, 4, 7, 9] on row 22
try to change, with changing index [0, 3, 5, 6, 9] on row 22
try to change, with changing index [0, 2, 6, 7, 8] on row 22
try to change, with chan

try to change, with changing index [1, 3, 9] on row 24
try to change, with changing index [3, 4, 8] on row 24
try to change, with changing index [0, 1, 4] on row 24
try to change, with changing index [2, 7, 9] on row 24
try to change, with changing index [1, 4, 6] on row 24
try to change, with changing index [0, 6, 8] on row 24
try to change, with changing index [2, 3, 8] on row 24
try to change, with changing index [3, 8, 9] on row 24
try to change, with changing index [0, 3, 6] on row 24
try to change, with changing index [0, 1, 2] on row 24
try to change, with changing index [1, 2, 6] on row 24
try to change, with changing index [0, 1, 9] on row 24
try to change, with changing index [0, 4, 8] on row 24
try to change, with changing index [1, 6, 9] on row 24
try to change, with changing index [4, 6, 8] on row 24
try to change, with changing index [0, 3, 4] on row 24
try to change, with changing index [3, 4, 6] on row 24
try to change, with changing index [1, 2, 4] on row 24
try to cha

try to change, with changing index [0, 2, 4, 6, 7, 8] on row 24
try to change, with changing index [0, 1, 2, 4, 5, 9] on row 24
try to change, with changing index [1, 2, 4, 5, 6, 9] on row 24
try to change, with changing index [0, 4, 6, 7, 8, 9] on row 24
try to change, with changing index [0, 2, 5, 6, 8, 9] on row 24
try to change, with changing index [2, 3, 4, 7, 8, 9] on row 24
try to change, with changing index [0, 1, 2, 3, 6, 8] on row 24
try to change, with changing index [0, 2, 3, 4, 6, 7] on row 24
try to change, with changing index [0, 1, 3, 6, 8, 9] on row 24
try to change, with changing index [0, 3, 4, 6, 7, 9] on row 24
try to change, with changing index [0, 1, 2, 4, 7, 9] on row 24
try to change, with changing index [0, 2, 3, 5, 6, 9] on row 24
try to change, with changing index [1, 2, 4, 6, 7, 9] on row 24
try to change, with changing index [0, 2, 6, 7, 8, 9] on row 24
try to change, with changing index [0, 2, 4, 5, 8, 9] on row 24
try to change, with changing index [0, 1

array([[  4.35000000e+02,  -3.99000000e+02,   4.09000000e+02,
          3.98000000e+02,  -2.60000000e+01,   6.08000000e+02,
          1.19000000e+02,   4.51000000e+02,  -2.10000000e+02,
          3.85000000e+02],
       [  3.00000000e+01,   2.00000000e+00,  -5.10000000e+01,
          3.10000000e+01,  -6.60000000e+01,  -1.80000000e+01,
         -2.10000000e+01,   1.50000000e+01,  -6.20000000e+01,
         -8.00000000e+00],
       [ -1.24000000e+02,  -4.00000000e+00,   3.10000000e+01,
         -2.40000000e+01,   1.10000000e+01,   2.00000000e+01,
          0.00000000e+00,  -4.20000000e+01,  -1.40000000e+01,
          1.90000000e+01],
       [  5.00000000e+01,  -2.60000000e+01,   4.20000000e+01,
          2.90000000e+01,   6.90000000e+01,   3.80000000e+01,
          5.70000000e+01,   4.10000000e+01,  -1.30000000e+01,
          4.70000000e+01],
       [  2.50000000e+01,  -1.07000000e+02,   2.00000000e+01,
         -1.77000000e+02,   3.40000000e+01,   1.40000000e+01,
          6.60000000e+01

In [ ]:
import heapq
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import chi2
from sklearn.ensemble import ExtraTreesClassifier
from accuracy_drop_proj.strategies.change_combination.change_combination import Change_Combination

"""
this  method first sort the rows according to uncetainity and features according to information gain and then pick
first rows that classifer is not sure about them and feature are more likely to change 
"""
class Change_Uncertainty_Rankfeatures(object):
    def __init__(self):
        pass

    def change(self,x_train, y_train, percetage, mnb, change_plan):
        number_change_requested = int(percetage / 100 * x_train.shape[0])
        print("{} percentage error is equal to {} change \n".format(percetage, number_change_requested))

        used_row ={}
        occurred_change = 0
        all_changed = 1
        change_done = False
        x_train_changed = np.copy(x_train)

        #---------------------find the order of the feature according to information gain-----------------------

        model = ExtraTreesClassifier()
        model.fit(x_train, y_train)

        print("combinatio of feature")

        information_gain = {}
        for i in range(len(model.feature_importances_)):
            information_gain.update({i: model.feature_importances_[i]})

        ranked_information_dic = {}
        sum_gain = 0
        for L in range(0,x_train.shape[1] + 1):
            for subset in Change_Combination.combinations_index(self,information_gain.keys(), L):
                if not subset:
                    pass
                else:

                    for key in subset:
                        sum_gain = sum_gain + information_gain.get(key)
                    ranked_information_dic.update({tuple(subset): sum_gain})
                    sum_gain = 0

        print("create all subset")

        all_subset = sorted(ranked_information_dic.items(), key=lambda item: len(item[0]) * 1000 - item[1], reverse=False)


        #---------------------------finding the order of the row according to uncertainity-------------------------

        probability = mnb.predict_proba(x_train)

        print("finding uncertainity")

        uncertainty={}
        for index,roww in enumerate(probability):
            largest_val =heapq.nlargest(2, roww)
            uncertainty.update({index:1-(np.abs(np.subtract(largest_val[0],largest_val[1])))})
            largest_val=[]
            # print(index,row,np.subtract(largest_val[0],largest_val[1]))

        print(len(probability))
        print(len(uncertainty))
        #sort the uncertainty
        uncertainty_sorted=sorted(uncertainty.items(), key=lambda x:x[1],reverse=True)


        print("changing")
        #---------------------------------------------changing--------------------------------------------

        for i in range(len(change_plan["key"])):
            occurred_change = 0
            #sort the row according to uncertainty

            indices=[]

            for key_dic in uncertainty_sorted:
                if y_train[key_dic[0]] == change_plan["key"][i][0]:
                    indices.append(key_dic[0])

            print(indices)


            #this is normal indices
            # indices_2 = [t for t, x in enumerate(y_train) if x == change_plan["key"][i][0]]


            print("{} rows have target {} \n".format(len(indices), change_plan["key"][i][0]))

            for p in range(len(indices)):
                print("try in indices")
                if (all_changed == number_change_requested + 1):
                    print("your requests have been done :)")
                    break
                if y_train[indices[p]] == mnb.predict([x_train[indices[p]]]) and indices[p] not in used_row:

                    change_done = False
                    for subset in all_subset:
                        if change_done:
                            break
                        else:

                            if (occurred_change == change_plan["number"][i]):
                                #                         print("part of your request has been done :))))")
                                break

                            print("try to change, with change index {} on row {}".format(list(subset[0]),indices[p]))
                            x_train_changed[indices[p]][list(subset[0])] = 0

                            if (change_plan["key"][i][1] == mnb.predict([x_train_changed[indices[p]]])[0]):

                                print(x_train[indices[p]], mnb.predict([x_train[indices[p]]])[0])
                                print(x_train_changed[indices[p]],
                                      mnb.predict([x_train_changed[indices[p]]])[0])
                                print(" \n change number {} \n".format(all_changed))

                                used_row.update({indices[p]: indices[p]})
                                occurred_change = occurred_change + 1
                                change_done = True
                                all_changed = all_changed + 1
                                break

                            else:
                                x_train_changed[indices[p]] = np.copy(x_train[indices[p]])

        if (all_changed <= number_change_requested):
            print("your request doesn't complete! please change your plan")
        return np.copy(x_train_changed)


In [ ]:
model2= Change_Uncertainty_Rankfeatures()
model2.change(x_test, y_test,10,model1, change_plan)